In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import sys
import os
import numpy as np
import cv2


sys.path.append(os.path.abspath(".."))

In [33]:
DATA_DIR  = ".\Application_programming_with_OpenCV_Streamlit\data\semantic_keypoint_detection\synthetic_shapes_datasets"

DATA_TYPES  = [
    os.path.join(DATA_DIR, "draw_checkerboard"),
    os.path.join(DATA_DIR, "draw_cube"),
    os.path.join(DATA_DIR, "draw_ellipses"),
    os.path.join(DATA_DIR, "draw_lines"),
    os.path.join(DATA_DIR, "draw_multiple_polygons"),
    os.path.join(DATA_DIR, "draw_polygon"),
    os.path.join(DATA_DIR, "draw_star"),
    os.path.join(DATA_DIR, "draw_stripes"),
]

In [9]:
def calculate_precision_recall(true_points: np.ndarray, predicted_points: np.ndarray):
    true_positive, false_positive, false_negative = 0, 0, 0

    # Duyệt qua từng điểm thực để kiểm tra có điểm dự đoán tương ứng gần đúng hay không
    for point in true_points:
        if predicted_points.shape[0] > 0 and np.any(
            np.linalg.norm(predicted_points - point, axis=1) <= 5
        ):
            true_positive += 1
        else:
            false_negative += 1

    # Duyệt qua từng điểm dự đoán để xác định các điểm không có điểm thực tương ứng gần đúng
    for pred_point in predicted_points:
        if true_points.shape[0] == 0 or not np.any(
            np.linalg.norm(true_points - pred_point, axis=1) <= 5
        ):
            false_positive += 1

    # Tính Precision và Recall
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return precision, recall

In [34]:

sift = cv2.SIFT_create()
orb = cv2.ORB_create()
# Hàm tính Precision và Recall cho một loại hình dạng
precision_recall_sift = []
precision_recall_orb = []

for i in range(8):
    pr_sift, pr_orb = [], []
    for j in range(500):
        # Tải điểm thực và ảnh từ thư mục tương ứng
        image = cv2.imread(os.path.join(DATA_TYPES[i], "images", f"{j}.png"))
        ground_truth = np.load(os.path.join(DATA_TYPES[i], "points", f"{j}.npy"))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        pred_sift = sift.detect(gray, None)
        pref_orb = orb.detect(gray, None)

        pred_sift = np.array([[kp.pt[1], kp.pt[0]] for kp in pred_sift])
        pref_orb = np.array([[kp.pt[1], kp.pt[0]] for kp in pref_orb])

        pr_sift.append(calculate_precision_recall(ground_truth, pred_sift))
        pr_orb.append(calculate_precision_recall(ground_truth, pref_orb))

    precision_recall_sift.append(pr_sift)
    precision_recall_orb.append(pr_orb)

In [36]:
results_dir = ".\Application_programming_with_OpenCV_Streamlit\data\semantic_keypoint_detection"

# Lưu kết quả Precision và Recall cho SIFT và ORB vào file .npy
np.save(os.path.join(results_dir, "precision_recall_sift.npy"), precision_recall_sift)
np.save(os.path.join(results_dir, "precision_recall_orb.npy"), precision_recall_orb)

# In kích thước của các mảng kết quả để kiểm tra
print("Kích thước của precision_recall_sift:", np.shape(precision_recall_sift))
print("Kích thước của precision_recall_orb:", np.shape(precision_recall_orb))


Kích thước của precision_recall_sift: (8, 500, 2)
Kích thước của precision_recall_orb: (8, 500, 2)
